
#Build, Train, and Test CNN on MNIST Dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:

import tensorflow as tf

In [ ]:

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

x = tf.placeholder(tf.float32, shape=[None, 784])
y_true = tf.placeholder(tf.float32, shape=[None, 10])
hold_prob = tf.placeholder(tf.float32)

x_image = tf.reshape(x, [-1, 28, 28, 1])

def initialize_weights(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def initialize_bias(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def create_convolution_layer_and_compute_dot_product(inputs, filter_shape):
    filter_initialized_with_weights = initialize_weights(filter_shape)
    conv_layer_output = tf.nn.conv2d(inputs, filter_initialized_with_weights, strides=[1, 1, 1, 1], padding='SAME')
    return conv_layer_output

def create_relu_layer_and_compute_dotproduct_plus_b(inputs, filter_shape):
    b = initialize_bias([filter_shape[3]])
    relu_layer_output = tf.nn.relu(inputs + b)
    return relu_layer_output

def create_maxpool2by2_and_reduce_spatial_size(inputs):
    pooling_layer_output = tf.nn.max_pool(inputs, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    return pooling_layer_output

def create_fully_connected_layer_and_compute_dotproduct_plus_bias(input, output_size):
    weights = initialize_weights([input.get_shape()[1], output_size])
    bias = initialize_bias([output_size])
    layer = tf.matmul(input, weights) + bias
    return layer

conv_layer_1_outputs = create_convolution_layer_and_compute_dot_product(x_image, filter_shape=[5, 5, 1, 32])
conv_relu_layer_1_outputs = create_relu_layer_and_compute_dotproduct_plus_b(conv_layer_1_outputs, filter_shape=[5, 5, 1, 32])
pooling_layer_1_outputs = create_maxpool2by2_and_reduce_spatial_size(conv_relu_layer_1_outputs)

conv_layer_2_outputs = create_convolution_layer_and_compute_dot_product(pooling_layer_1_outputs, filter_shape=[5, 5, 32, 64])
conv_relu_layer_2_outputs = create_relu_layer_and_compute_dotproduct_plus_b(conv_layer_2_outputs, filter_shape=[5, 5, 32, 64])
pooling_layer_2_outputs = create_maxpool2by2_and_reduce_spatial_size(conv_relu_layer_2_outputs)
pooling_layer_2_outputs_flat = tf.reshape(pooling_layer_2_outputs, [-1, 7*7*64])


fc_layer_1_outputs = create_fully_connected_layer_and_compute_dotproduct_plus_bias(pooling_layer_2_outputs_flat, output_size=1024)
fc_relu_layer_1_outputs = tf.nn.relu(fc_layer_1_outputs)

fc_dropout_outputs = tf.nn.dropout(fc_relu_layer_1_outputs, keep_prob=hold_prob)


y_pred = create_fully_connected_layer_and_compute_dotproduct_plus_bias(fc_dropout_outputs, output_size=10)

softmax_cross_entropy_loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)
cross_entropy_mean = tf.reduce_mean(softmax_cross_entropy_loss)
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
cnn_trainer = optimizer.minimize(cross_entropy_mean)

vars_initializer = tf.global_variables_initializer()

steps = 5000

with tf.Session() as sess:
    sess.run(vars_initializer)

    for i in range(steps):
        batch_x, batch_y = mnist.train.next_batch(50)
        sess.run(cnn_trainer, feed_dict={x: batch_x, y_true: batch_y, hold_prob: 0.5})

        if i % 100 == 0:
            print('ON STEP: {}'.format(i))
            print('ACCURACY:')
            matches = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
            acc = tf.reduce_mean(tf.cast(matches, tf.float32))
            test_accuracy = sess.run(acc, feed_dict={x: mnist.test.images, y_true: mnist.test.labels, hold_prob: 1.0})
            print(test_accuracy)
            print('\n')


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
ON STEP: 0
ACCURACY:
0.1033


ON STEP: 100
ACCURACY:
0.9395


ON STEP: 200
ACCURACY:
0.9593


ON STEP: 300
ACCURACY:
0.9712


ON STEP: 400
ACCURACY:
0.973


ON STEP: 500
ACCURACY:
0.9758


ON STEP: 600
ACCURACY:
0.9779


ON STEP: 700
ACCURACY:
0.9744


ON STEP: 800
ACCURACY:
0.9817


ON STEP: 900
ACCURACY:
0.9824


ON STEP: 1000
ACCURACY:
0.9835


ON STEP: 1100
ACCURACY:
0.9847


ON STEP: 1200
ACCURACY:
0.9862


ON STEP: 1300
ACCURACY:
0.9852


ON STEP: 1400
ACCURACY:
0.9844


ON STEP: 1500
ACCURACY:
0.9859


ON STEP: 1600
ACCURACY:
0.9849


ON STEP: 1700
ACCURACY:
0.9864


ON STEP: 1800
ACCURACY:
0.9844


ON STEP: 1900
ACCURACY:
0.9868


ON STEP: 2000
ACCURACY:
0.9867


ON STEP: 2100
ACCURACY:
0.9846


ON STEP: 2200
ACCURACY:
0.9867


ON STEP: 2300
ACCURACY:
0.9878


ON STEP: 2400
ACCURACY:
0.9